[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/prokaj/elte-python-2023/blob/main/2023-10-02.ipynb)   

In [1]:
import importlib

if importlib.util.find_spec('ipytest') is None:
    ! pip install ipytest

import ipytest

ipytest.autoconfig()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.7 MB/s eta 0:00:00


# Házi feladatok



## Feladat



Írjunk három függvényt.

Az első (`poly_add`) két együttható sorozattal (listával) adott polinomot ad össze. A függvény értéke a két polinom összegének együttható sorozata.

A második (`poly_mul`) két együttható sorozattal (listával) adott polinomot összeszoroz. A függvény értéke a két polinom szorzatának együttható sorozata.

A harmadik (`poly_norm`) a gyakorlaton is használt $p_0≡1$, $p_1(x)=x$, $p_k(x)=\frac1{k!}x(x+1)⋯(x+k−1)$ bázisról tér át a természetes bázisra. Azaz $c_0,\dots,c_r$ együttható sorozatból kiszámítja az $a_0,\dots,a_r$ sorozatot úgy, hogy
$$
r!\sum_{k=0}^ r c_kp_k(x) = \sum_{k=0}^r a_kx^k
$$
Az $r!$-nak csak annyi a szerepe, hogy egész értékű $c_0,\dots,c_r$ esetén az $a_0,\dots,a_r$ értékek is egészek legyenek!

In [5]:
import math

def poly_add(a, b):
    if len(a) > len(b):
        a, b = b, a
    result = b.copy()
    for i in range(len(a)):
        result[i] += a[i]
    # for i, ai in enumerate(a):
    #     result[i] += ai
    while len(result) > 1 and result[-1] == 0:
        result.pop()

    return result

def poly_mul(a, b):
    result = [0]*(len(a)+len(b)-1)
    for i in range(len(a)):
        for j in range(len(b)):
            result[i+j] += a[i]*b[j]
    return result

def poly_norm(coeff):

    p = [1]
    result = [0]

    r_factorial = math.factorial(len(coeff)-1)

    for i, ci in enumerate(coeff):
        result = poly_add(result, poly_mul(p, [ci*r_factorial]))
        p = poly_mul(p, [i, 1])
        r_factorial //= (i+1)

    return result

In [13]:
poly_norm([0, -1, 2])

[0, 0, 2]

Néhány segédfüggvény.

In [3]:
def poly_taylor(coeff):
    """
    returns a polinomial function
    sum coeff[i] x**i = coeff[0] + x*(coeff[1] + x*(...))
    """
    def f(x):
        value = 0
        for c in reversed(coeff):
            value = value*x+c
        return value

    return f

def poly_other(coeff):
    """
    returns a polinomial function
    sum coeff[i] x*(x+1)*...(x+i-1) = coeff[0] + x*(coeff[1] + (x+1)*(...))
    """
    def f(x):
        value = 0
        p = 1
        for i, c in enumerate(coeff):
            value += c*p
            p = p*(x+i)/(i+1)
        return value

    return f

In [14]:
%%ipytest

def test_poly_add():
    a = [1, 2, 3]
    b = [0,1,0,1,0,1]
    c = poly_add(a, b)
    pa = poly_taylor(a)
    pb = poly_taylor(b)
    pc = poly_taylor(c)
    for x in range(max(len(a), len(b), len(c))):
        assert pc(x) == pa(x) + pb(x)

def test_poly_mul():
    a = [1, 2, 3]
    b = [0,1,0,1,0,1]
    c = poly_mul(a, b)
    pa = poly_taylor(a)
    pb = poly_taylor(b)
    pc = poly_taylor(c)
    for x in range(max(len(a), len(b), len(c))):
        assert pc(x) == pa(x) * pb(x)


def test_poly_norm():
    a = [1, 2, 3]
    b = poly_norm(a)
    pa = poly_other(a)
    pb = poly_taylor(b)
    r_factorial = math.factorial(len(a)-1)
    for x in range(max(len(a), len(b))):
        assert r_factorial*pa(x) == pb(x)


...                                                                                          [100%]
3 passed in 0.02s


## Szorgalmi feladat


Adott egy számokból álló $n\geq 1$ hosszú lista és egy pozitív egész $k$, $1\leq k\leq n$.

A lista elemeit $a_0,a_1,\dots,a_{n−1}$ jelöli. Írjunk egy függvényt, ami minden $0\leq i < n-k$ indexre kiszámítja az  $a_i,a_{i+1},\dots,a_{i+k−1}$ értékek maximumát.

A függvény eredménye tehát egy $n−k+1$ hosszú lista, aminek $i$-k eleme: $\max_{i\leq j<i+k} a_j$.

Írjuk meg a függvény naiv, és ezért egyszerű változatát. Nem számít, ha nem optimális, de legyen átlátható és ezt is teszteljük néhány nem túl hosszú sorozaton.

Plusz 2 pontért írjuk meg az optimalizált változatot is, ami $O(n)$
lépésben számolja az eredményt. Az optimalizált változatot teszteljük a naiv változat segítségével.

Végül plusz 1 pont a lineáris futási idő bizonyításáért.



In [15]:
def window_maxes(nums, k):
    return [max(nums[i:i+k]) for i in range(len(nums)-k+1)]


In [16]:
%%ipytest
import random

def test_window_maxes_short_list():
    lst = [0, 1, 2,-3, 4, -1]
    assert window_maxes(lst, 1) == lst
    assert window_maxes(lst, 2) == [1, 2, 2, 4, 4]
    assert window_maxes(lst, 3) == [2, 2, 4, 4]
    assert window_maxes(lst, 4) == [2, 4, 4]
    assert window_maxes(lst, 5) == [4, 4]
    assert window_maxes(lst, 6) == [4]

def pair_maxes(lst):
    a = lst[0]
    result = []
    for b in lst[1:]:
        result.append(max(a, b))
        a = b
    return result

def test_window_maxes_random_list():
    lst = [random.randint(-1000, 1000) for _ in range(100)]
    max_lst = lst.copy()
    for k in range(1, 10):
        assert window_maxes(lst, k) == max_lst
        max_lst = pair_maxes(max_lst)

..                                                                                           [100%]
2 passed in 0.01s


In [17]:
def window_maxes_fast(lst, k):

    idx = 0
    running_maxes = []
    result = []
    for i in range(k):
        x = lst[i]
        while running_maxes and running_maxes[-1] < x:
            running_maxes.pop()
        running_maxes.append(x)

    result.append(running_maxes[idx])
    for i in range(k, len(lst)):
        if lst[i-k] == running_maxes[idx]:
            idx += 1
        x = lst[i]
        while len(running_maxes) > idx and running_maxes[-1] < x:
            running_maxes.pop()
        running_maxes.append(x)
        result.append(running_maxes[idx])
    return result


In [18]:
%%ipytest

def test_window_maxes_fast():
    lst = [random.randint(-1000, 1000) for _ in range(100)]
    for k in range(1, 10):
        assert window_maxes(lst, k) == window_maxes_fast(lst, k)

.                                                                                            [100%]
1 passed in 0.01s


Gyorsabb-e az optimalizált változat?

In [19]:
lst = [random.randint(-1000, 1000) for _ in range(1000)]
%timeit window_maxes(lst, len(lst)//2)
%timeit window_maxes_fast(lst, len(lst)//2)

5.48 ms ± 87.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
423 µs ± 5.59 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# Mi kerülhet a `for` ciklusban az `in` mögé?



Tulajdonképpen bármi, amin végig lehet iterálni. Mit jelent ezt?

```Python
for x in seq:
    do_something_with(x)
```
        
nagyjából a következőt jelenti:

```Python
it = iter(seq)
while True:
    x = next(it)
    do_something_with(x)
```

hogyan lépünk ki a ciklusból.  Ha sorozat végére érünk az iterátor egy `StopIteration` kivételt dob, aminek hatására a ciklus félbeszakad.

In [20]:
it = iter("kutya")
print(it)
print(next(it)) # -> k
print(next(it)) # -> u
print(next(it)) # -> t
print(next(it)) # -> y
print(next(it)) # -> a
print(next(it)) # -> ?



k
u
t
y
a


StopIteration: ignored

Tegyük fel, hogy a sorozatban azon elemek száma érdekel minket, amelyek az előzőtől eltérnek.

In [21]:
seq = map(int, "1 1 2 2 3 4 5 5 5".split())
print(f"{seq = }")
it = iter(seq)
a = next(it)
cnt = 1
for b in it:
    if b != a:
        cnt += 1
    a = b
print(f"különböző elemek száma: {cnt}")

seq = <map object at 0x7f58aa70dc00>
különböző elemek száma: 5


Próbáljuk ugyanezt indexeléssel megoldani.

In [22]:

for seq in [list(range(10)), range(10), map(int, "1 1 2 2 3 4 5 5 5".split())]:

    print(f"{seq = }")

    a = seq[0]
    cnt = 1

    for b in seq:
        if b != a:
            cnt += 1
        a = b

    print(f"különböző elemek száma: {cnt}")
    print("="*50)

seq = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
különböző elemek száma: 10
seq = range(0, 10)
különböző elemek száma: 10
seq = <map object at 0x7f58aa70da80>


TypeError: ignored

In [23]:
for seq in [list(range(10)), range(10), map(int, "1 1 2 2 3 4 5 5 5".split())]:

    print(f"{seq = }")
    it = iter(seq)
    print(f"{it = }")

    a = next(it)
    cnt = 1
    for b in it:
        if b != a:
            cnt += 1
        a = b

    print(f"különböző elemek száma: {cnt}")
    print(f"{list(it) = }")
    print("="*50)

seq = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
it = <list_iterator object at 0x7f58aa70e7a0>
különböző elemek száma: 10
list(it) = []
seq = range(0, 10)
it = <range_iterator object at 0x7f58aa70daa0>
különböző elemek száma: 10
list(it) = []
seq = <map object at 0x7f58aa70c0d0>
it = <map object at 0x7f58aa70c0d0>
különböző elemek száma: 5
list(it) = []


## Sorozatokból újabb sorozatok

A leggyakrabban használt függvények:

- `enumerate`

- `reversed`

- `zip`

### Példák

In [24]:
a = range(1, 10)
b = "kutya"

print(f"{enumerate(b) = },\n{list(enumerate(b)) = }\n====")

print(f"{reversed(a) = },\n{list(reversed(a)) = }\n====")

print(f"{zip(a,b) = },\n{list(zip(a, b)) = }")


enumerate(b) = <enumerate object at 0x7f58aa66afc0>,
list(enumerate(b)) = [(0, 'k'), (1, 'u'), (2, 't'), (3, 'y'), (4, 'a')]
====
reversed(a) = <range_iterator object at 0x7f58aa7351d0>,
list(reversed(a)) = [9, 8, 7, 6, 5, 4, 3, 2, 1]
====
zip(a,b) = <zip object at 0x7f58aa836b40>,
list(zip(a, b)) = [(1, 'k'), (2, 'u'), (3, 't'), (4, 'y'), (5, 'a')]


Vegyük észre, hogy a `zip`-elt sorozat hossza a **rövidebb** sorozat hosszával egyezik meg!

In [26]:
for i, ch in enumerate("kutya", 1):
    print(f"az {i}-ik karakter {ch}")

az 1-ik karakter k
az 2-ik karakter u
az 3-ik karakter t
az 4-ik karakter y
az 5-ik karakter a


In [27]:
''.join(reversed("kutya"))

'aytuk'

Mátrix transzponálás

In [28]:
matrix = [[1,2,3], [3,4,5]]
print(*matrix, sep="\n")
transposed_matrix = [list(x) for x in zip(*matrix)]
print(*transposed_matrix, sep="\n")


[1, 2, 3]
[3, 4, 5]
[1, 3]
[2, 4]
[3, 5]


## Saját sorozatok


Ha egy függvényben `yield` szerepel, akkor egy generátor függvényt kapunk.

Példa.

In [29]:
def natural_nums(n):
    for i in range(1, n+1):
        yield i

In [30]:
print(f"{natural_nums=}\n{natural_nums(10)=}\n{iter(natural_nums(10))=}")

natural_nums=<function natural_nums at 0x7f58aa588d30>
natural_nums(10)=<generator object natural_nums at 0x7f58aa57d460>
iter(natural_nums(10))=<generator object natural_nums at 0x7f58aa57d460>


A függvényhívás eredménye egy generátor objektum, az `iter` függvénnyel szintén ezt kapjuk, végül a `next` függvény veszi a sorozat következő elemét, ha még van.



In [31]:
print(next(natural_nums(10)))
print(next(natural_nums(10)))
print(next(natural_nums(10)))



1
1
1


Miért kapjuk mindig az `1`-et?

In [32]:
it = natural_nums(10)
print(next(it))
print(next(it))
print(next(it))
print(next(it))


1
2
3
4


Azt mondtuk, hogy az iterációnak a `StopIteration` kivétel vet véget. Tegyük fel, hogy szeretnénk jelezni, hogy vége van a sorozatnak.

Kell-e

```
raise StopIteration
```

**Nem**

Elég egy `return`


### Faktoriálisok sorozata

In [33]:
def factorials(n):
    a = 1
    ## 0!
    yield a
    for i in range(1, n+1):
        a *= i
        yield a


In [34]:

print(factorials, factorials(3))
print(list(factorials(3)))


<function factorials at 0x7f58aa589630> <generator object factorials at 0x7f58aa57d700>
[1, 1, 2, 6]


In [35]:

for f in factorials(3):
    print(f, end=" ")
print()


1 1 2 6 


In [37]:

it = iter(factorials(4))
print(it, next(it), next(it), next(it), next(it))
next(it)

<generator object factorials at 0x7f58aa57d700> 1 1 2 6


24

Az első $n$, amire $n!>1\_000\_000\_000$

In [39]:
def factorials(n=None):
    a = 1
    i = 1
    ## 0!
    yield a
    while (n is None) or (i <= n):
        a *= i
        yield a
        i += 1


In [40]:

threshold = 1_000_000_000
for i, f in enumerate(factorials()):
    print(f"{i:3} faktoriális: {f:14_}")
    if f > threshold:
        break
print(f"{i}, {f:_}")

  0 faktoriális:              1
  1 faktoriális:              1
  2 faktoriális:              2
  3 faktoriális:              6
  4 faktoriális:             24
  5 faktoriális:            120
  6 faktoriális:            720
  7 faktoriális:          5_040
  8 faktoriális:         40_320
  9 faktoriális:        362_880
 10 faktoriális:      3_628_800
 11 faktoriális:     39_916_800
 12 faktoriális:    479_001_600
 13 faktoriális:  6_227_020_800
13, 6_227_020_800


# Függvényekről újra

## Láthatóság



Példa


In [ ]:
def plus_one():
    return new_variable+1

In [ ]:

plus_one() # -> Error

In [ ]:
new_variable = 1
plus_one() # -> 2

esetén, `new_variable` egy globális valtozó.

Globális változókat csak indokolt esetben használjunk, pl. konstansok tárolására!

Bármi, aminek a függvény belsejében értéket adunk lokális változó lesz.


In [ ]:
def plus_one():
    new_variable_ += 1
    return new_variable_

In [ ]:
plus_one() # -> Error

In [ ]:
new_variable_ = 1
plus_one() # -> Error



Mindkét esetben hibát kapunk de nem ugyanazt mint az előbb!! Mi a különbség?

### Függvényen belüli függvény definíció

Találjuk ki mi a visszatérési érték az utolsó sorban!
```Python
a = 1

def f1(x)
    a = x
    def g()
        return a
    return g

def f2(a)
    def g()
        return a
    return g

f1(3)(), f2(4)(), a
```

### Mutable típusú változók

Találjuk ki mi a visszatérési érték az utolsó sorban!
```Python
a = [1]
    
def f1(x):
    a[0] = x
    def g():
        return a[0]
    return g

def f2(x):
    a = [x]
    def g():
        a[0] += 1
        return a[0]
    return g

g2 = f2(4)
f1(3)(), g2(), g2(), a
```


## Függvény változó számú paraméterrel

Szintakszis:


In [ ]:
def f(*args):
    print(f"args típusa: {type(args).__name__}, {args = !r}")


Mikor lehet rá szükség? pl. ha a paraméterek száma előre nem ismert. A `print` függvény ilyen, a `max` és `min` függvény is.

In [ ]:
print(f"{max(1, 2, 3)=}, {min(1, 2, 3)=}")
print(1,2,3)

In [ ]:
f(1, 2, 2, 3)

In [ ]:
f("kutya")

In [ ]:
f(range(10))

Mi van, ha a paraméterek már eleve egy sorozatban, pl. listában, vagy sztringben vannak?

Ha az argumentum előtt `*` van, akkor a Python szétszedi a sorozatot.

In [ ]:
def g(a, b):
    print(f"{a=}, {b=}")

In [ ]:
g(*["alma", 1])

A paraméterek számának egyezni kell:

In [ ]:
g(*[1,2,3])

A `*`-ot olyan függvénnyel is használhatjuk, mint `f`

In [ ]:
f(*range(5))

In [ ]:
f(*"kutya")

In [ ]:
f(*[1,2,3])

## `*args` mellett lehet más is?

Igen. de mi történik a következő esetekben?

In [ ]:
def f(a, *args, b):
    print(f"{a=}, {args=}, {b=}")

In [ ]:
f(1, 2, 3, 4)

In [ ]:
f(1, 2, 3, b=3)

Azt is kikényszeríthetjük, hogy bizonyos paramétereket névvel kelljen megadni.

In [ ]:
def f(*, a, b):
    print(f"{a=}, {b=}")


In [ ]:
a = 1
b = 2
f(a, b)

In [ ]:
f(a=a, b=b)

# Polinomok

## Feladat

Írjunk egy függvényt, ami kiszámolja az első $n$ természetes szám $p$-ik hatványösszegét.

pl. `p = 0`-ra

```Python
def f0(n):
    return n
```

jó, mert $k^0=1$ ha $k=1,\dots,n$ és ezek összege pont $n$.

Ha `p = 1`, akkor

```Python
def f1(n):
    return n*(n+1)//2
```

jó, mert $\sum_{k=1}^n k = n(n+1)/2$.

Még `p = 2`-t is tanultuk

```Python
def f2(n):
    return n*(n+1)*(2*n+1)//6
```

Általános $p$-re tudunk-e ilyen függvényt írni?

In [ ]:
is_prime_naive.__annotations__

In [ ]:
def mk_power_sum(p):
    def f(n):
        total = 0
        for k in range(1, n+1):
            total += k**p
        return total

    f.__doc__ = f"""
        {p}-ik hatványok összegét számolja
        """

    return f

In [ ]:
f2_slow = mk_power_sum(2)

In [ ]:
f2_slow?

In [ ]:
[f2_slow(i) for i in range(0, 10)]

In [ ]:
def f2_fast(n):
    return n*(n+1)*(2*n+1)//6

In [ ]:
%timeit f2_slow(10_000)
%timeit f2_fast(10_000)

### Ötlet

$$
    \sum_{k=r}^n  \binom{k}{r} = \binom{n+1}{r+1}
$$

**Bizonyítás.**
$\{1,2,\dots,n+1\}$-ből válasszunk ki $r+1$ különböző számot.

Összes lehetőség:
$$
\binom{n+1}{r+1}.
$$

Számoljuk meg az eseteket aszerint szétbontva is, hogy legnagyobb kiválasztott szám mivel egyenlő.

Ha a legnagyobb szám $k+1$, akkor a maradék $r$ számot $\{1,2,\dots, k\}$ közül választjuk. Így az esetek száma
$$
    \sum_{k+1=r+1}^{n+1} \binom{k}{r} =  \sum_{k=r}^{n} \binom{k}{r}
$$  
$k+1$ helyett $k$ az összegzési változó

Ugyanez másképp.

$$
\binom{k}{r} = \frac{1}{r!} k(k-1)\cdots(k-r+1) = p_r(k-r+1),\quad\text{ahol}\quad p_r(x) = \frac{x(x+1)\cdots(x+r-1)}{r!}
$$
és
$$
    \sum_{j=1}^{n-r+1} p_r(j) = p_{r+1}(n+1-(r+1)+1)= p_{r+1}(n-r+1)\quad\text{minden $n\geq r$ és $r\geq 0$-ra}
$$

Az összegzés felső határa és $p_{r+1}$ argumentuma ugyanaz, azaz

$$
\sum_{j=0}^{n} p_r(j) = p_{r+1}(n)
$$

**Lineáris algebra.**

$$
p_0\equiv 1,\quad p_1(x)=x,\quad p_2(x)=\frac12x(x+1),\quad\dots,\quad p_r(x)=\frac1{r!}x(x+1)\cdots(x+r-1)
$$

bázis a legfeljebb $r$-edfokú polinomok vektorterében.

$$
    x^r = \sum_{i=0}^r a_i p_i(x)
$$
és
$$
    \sum_{k=0}^n k^r = \sum_{k=0}^n \sum_{i=0}^r a_i p_i(k) =  \sum_{i=0}^r a_i  \sum_{k=0}^n p_i(k) = \sum_{i=0}^r a_i  p_{i+1}(n)
$$


### Összefoglalva

- Egy polinomot az együtthatókkal ábrázolhatunk.
- Kellene egy függvény, ami a természetes $1, x, x^2,\dots$ bázisban felírt polinomot a $p_0,p_1,\dots$ bázisban ír fel.
- $p_0, p_1, \dots,$ bázisban az összegzés könnyű, lényegében arrébb kell tolni az együtthatókat.
- A $p_0,p_1,\dots$ bázisban felírt polinomot vagy visszaszámoljuk a természetes bázisba, vagy megírjuk a függvényt, ami kiértékeli a függvényt egy adott pontban.

In [ ]:
def poly_taylor(coeff):
    """
    returns a polinomial function
    sum coeff[i] x**i = coeff[0] + x*(coeff[1] + x*(...))
    """
    def f(x):
        value = 0
        for c in reversed(coeff):
            value = value*x+c
        return value

    return f

def poly_other(coeff):
    """
    returns a polinomial function
    sum coeff[i] x*(x+1)*...(x+i-1) = coeff[0] + x*(coeff[1] + (x+1)*(...))
    """
    def f(x):
        value = 0
        p = 1
        for i, c in enumerate(coeff):
            value += c*p
            p = p*(x+i)/(i+1)
        return value

    return f

In [ ]:
f = poly_taylor([0, 0, 1])

f(0), f(1), f(2), f(0.5)


In [ ]:
g = poly_other([0, 0, 1])

g(0), g(1), g(2), g(0.5)


Vegyük észre, hogy $p_0$ azonosan 1, $p_1(0)=0$, $p_2(0)=p_2(-1)=0$, stb.

Ha $f=\sum_i a_i p_i$, akkor
$$
    f(0) = \sum_i a_i p(0) = a_0, \quad f(-1) = a_0 p_0(-1) + a_1 p_1(-1),\quad f(-k) = a_0 p_0(-k) + a_1 p_1(-k) + \cdots + a_k p_k(-k).
$$
amiből

$$
\begin{align*}
    a_0 & = f(0)\\
    a_1 & = \frac{f(-1) - a_0 p_0(-1)}{p_1(-1)}\\
    \vdots\\
    a_k & = \frac{f(-k) - \sum_{j=0}^{k-1} a_j p_j(-k)}{p_{k}(-k)}\\
    \vdots
\end{align*}
$$  
Kihasználhatjuk még, hogy
$$
p_k(-k)=\frac{1}{k!}(-k)(-k+1)\cdots(-k+(k-1))=(-1)^k.
$$

In [ ]:
def taylor_to_other(coeff):
    f = poly_taylor(coeff)
    new_coeff = []
    def p_value(x):
        value = 0
        p = 1
        for i, c in enumerate(new_coeff):
            value += c*p
            p = (p*(x+i))//(i+1)
        return value

    for i in range(len(coeff)):
        c = f(-i)-p_value(-i)
        if i % 2 == 1:
            c = -c
        new_coeff.append(c)

    return new_coeff

In [ ]:
taylor_to_other([0, 0, 0, 1])

Gyors ellenőrzés.

In [ ]:
coeff = [0,0,0,1]
f0 = poly_taylor(coeff)
f1 = poly_other(taylor_to_other(coeff))

for i in range(10):
    print(f"{i=}, {f0(i)=}, {f1(i)=}")

In [ ]:
def mk_fast_power_sum(p):
    coeff = taylor_to_other([0]*p+[1])
    def f(n: int) -> int:
        value = 0
        px = n
        for i, c in enumerate(coeff, 1):
            value += c*px
            px = (px*(n+i))//(i+1)
        return value


    f.__doc__ = f"""
    Computes the sum of the p-th power of the first `n` positive integer with {p = }.
    """
    return f

In [ ]:
taylor_to_other([0]*2+[1])

In [ ]:
f2 = mk_fast_power_sum(2)

In [ ]:
f2?

In [ ]:
f2(0), f2(1), f2(2), f2(3)

In [ ]:
%%ipytest

def test_fast_power_sum():
    for i in range(1, 4):
        f = mk_fast_power_sum(i)
        g = mk_power_sum(i)
        for n in range(1000):
            assert f(n) == g(n)

In [ ]:
%timeit f2_slow(10_000)
%timeit f2(10_000)
%timeit f2_fast(10_000)

In [ ]:
cell = f2.__closure__[0]
cell.cell_contents

# Egy gráfelméleti algoritmus

## Feladat

Adott egy `n` csúcsú irányítatlan gráf az élek listájával. A gráf csúcsait `0`-tól `n-1`-ig címkéztük meg, az éleket pedig a végpontokkal.

Emellett adott egy kiindulási pont és egy végpont. Azt szeretnénk eldönteni, hogy el lehet-e jutni a kiindulási pontból a végpontba a gráf éleit használva.

Gondolhatunk arra, hogy a gráf egy úthálózatot ír le és a kérdés az, hogy el tudunk-e jutni `A`-ból `B`-be.

Pl. `n = 3`, élek `edges = [[0,1], [1,2], [2,0]]`, `A =  0`, `B = 2`.

Gráfok megjelenítésére egy hasznos könyvtár a `graphviz`.

In [ ]:
import importlib
if importlib.util.find_spec('graphviz') is None:
    ! pip install graphviz
import graphviz

In [ ]:
edges = [[0,1], [1,2], [2,0]]
g0 = graphviz.Graph()

g0.edges([(str(a), str(b)) for a, b in edges])
g0

A gráf összefüggő, tetszőleges `A`, `B` esetén a válasz: `True`

In [ ]:
n = 6
edges = [[0,1],[0,2],[3,5],[5,4],[4,3]]
A = 0
B = 5

g1 = graphviz.Graph()
g1.edges([(str(a), str(b)) for a, b in edges])
g1

Nincs út 0 és 5 között. A válasz: `False`

Összefüggőségi komponenseket szeretnénk számolni.



## Ötlet.

Az él nélküli gráfból indulunk ki. Itt egy elemű komponensek vannak.

Minden komponensből válasszunk egy reprezentáns és minden $i$ pontra feljegyezzük, melyik komponensben van.




In [ ]:
def show_graph(roots, direction = 'LR'):
    g = graphviz.Digraph(graph_attr={'rankdir': direction})
    g.edges((str(i), str(r)) for i, r in enumerate(roots))
    return g

In [ ]:
n = 5
roots = [i for i in range(n)]
display(show_graph(roots, 'TD'))

ha behúzzuk a $(0, 1)$ élet, akkor $0$ és $1$ azonos komponensbe kerül. Választhatunk a két összeuniózott komponens reprezentánsa között, legyen pl. 1

In [ ]:
roots[0] = 1
display(show_graph(roots, 'TD'))

Ha most a (0, 2) élet akarjuk behúzni, akkor nem állíthatjuk át `roots[0]`. Meg kell keresnünk `0` komponensének reprezentását, ez 1 és vagy `roots[1]`-et állítjuk 2 -re, vagy `roots[2]`-t 1-re.

In [ ]:
def find(roots, a):
    while a != roots[a]:
        a = roots[a]
    return a

In [ ]:
find(roots, 0), find(roots, 2)

In [ ]:
def union(roots, a, b):
    ra = find(roots, a)
    rb = find(roots, b)
    roots[ra] = rb

In [ ]:
union(roots, 0, 2)
print(f"After union(0, 2) {roots=}")
display(show_graph(roots))

union(roots, 3, 4)
print(f"After union(3, 4) {roots=}")
print(roots)
display(show_graph(roots))

union(roots, 3, 2)
print(f"After union(3, 2) {roots=}")
print(roots)
display(show_graph(roots))

Ezután az a kérdés, hogy el lehet-e jutni `A`-ból, `B`-be könnyen eldönthető. Ha `A` és `B` azonos komponensben van, akkor `A` és `B` között megy út az eredeti gráfban, különben nem.

1. példa
`n = 3`, élek `edges = [[0,1], [1,2], [2,0]]`, `A =  0`, `B = 2`.

In [ ]:
def show_edges(edges, direction='LR'):
    g = graphviz.Graph(graph_attr={'rankdir': direction})
    g.edges([(str(a), str(b)) for a, b in edges])
    return g

In [ ]:
n = 3
edges = [[0,1], [1,2], [2,0]]
A =  0
B = 2

display(show_edges(edges, 'TD'))
print(f"Eredeti gráf")
print("="*50)

roots = [i for i in range(n)]
for a, b in edges:
    union(roots, a, b)

display(show_graph(roots))
print(f"{A=} és {B=} {'azonos' if find(roots, A)==find(roots, B) else 'különböző'} komponensben van")

2. példa

In [ ]:
n = 6
edges = [[0,1],[0,2],[3,5],[5,4],[4,3]]
A = 0
B = 5

display(show_edges(edges))
print(f"Eredeti gráf")
print("="*50)

roots = [i for i in range(n)]
for a, b in edges:
    union(roots, a, b)

display(show_graph(roots))
print(f"{A=} és {B=} {'azonos' if find(roots, A)==find(roots, B) else 'különböző'} komponensben van")


Mi történik, ha nagyobb gráfunk van?

In [ ]:
n = 10
edges = [(0,i+1) for i in range(n-1)]
display(show_edges(edges, 'TD'))

roots = [i for i in range(n)]
for a, b in edges:
    union(roots, a, b)

display(show_graph(roots))


Valahányszor behúzzuk a $(0, i)$ élet, meg kell keresni $0$ reprezentánsát. $k$ él behúzása után $k$-lépéssel találjuk meg. Ha 10 helyett 10_000 méretű a gráf ez nem fog működni.

### Javítási lehetőségek.

- Amikor megkeressük $i$ reprezentánsát végig megyünk a reprezentánshoz vezető úton. Minden meglátogatott csúcsra ismerté válik a reprezentáns értéke. Ezt beírhatjuk a `roots` tömbe. (path compression)

- A nagyobb komponensbe kössük be a kisebbet és ne fordítva. Ehhez  a ,,méretet'' nyilván kell tartani.

In [ ]:
def find_better(roots, a):
    ra = roots[a]
    if a != ra:
        ra = find_better(roots, ra)
        roots[a] = ra
    return ra

def find_better_without_recursion(roots, a):
    stack = []

    ra = roots[a]
    while a != ra:
        stack.append(a)
        a = ra
        ra = roots[a]

    while stack:
        roots[stack.pop()] = ra

    return ra


def union_sizes(roots, sizes, a, b):
    ra = find_better(roots, a)
    rb = find_better(roots, b)
    if ra != rb:
        if sizes[ra] < sizes[rb]:
            ra, rb = rb, ra
        roots[rb] = ra
        sizes[ra] += sizes[rb]

def union_ranks(roots, ranks, a, b):
    ra = find_better(roots, a)
    rb = find_better(roots, b)
    if ra != rb:
        if ranks[ra] < ranks[rb]:
            ra, rb = rb, ra
        roots[rb] = ra
        if ranks[ra] == ranks[rb]:
            ranks[ra] += 1




In [ ]:
n = 10
edges = [(0, i) for i in range(1, n)]
display(show_edges(edges, 'TD'))

roots = [i for i in range(n)]

for a, b in edges:
    ra = find_better(roots, a)
    rb = find_better(roots, b)
    roots[ra] = rb

display(show_graph(roots, 'LR'))


In [ ]:
n = 10
edges = [(0, i) for i in range(1, n)]
display(show_edges(edges, 'TD'))

roots = [i for i in range(n)]

for a, b in edges:
    ra = find_better(roots, a)
    rb = find_better(roots, b)
    roots[rb] = ra

display(show_graph(roots, 'TD'))


In [ ]:
n = 10
edges = [(0, i) for i in range(1, n)]
display(show_edges(edges, 'TD'))

roots = [i for i in range(n)]
sizes = [1]*n

for a, b in edges:
    union_sizes(roots, sizes, a, b)

display(show_graph(roots, 'TD'))


In [ ]:
n = 10
edges = [(0, i) for i in range(1, n)]
display(show_edges(edges, 'TD'))

roots = [i for i in range(n)]
ranks = [0]*n

for a, b in edges:
    union_ranks(roots, ranks, a, b)

display(show_graph(roots, 'TD'))


# Faktoriális értékének közelítése

Mekkora $n!$, ha $n$ nagy?

Ötlet:
$$
    \log n! = \sum_{k=1}^n \log k \approx \int_1^{?} \log x dx = \left[ x(\log x-1)\right]_{x=1}^{x=?}
$$

In [ ]:
import matplotlib.pyplot as plt
import math

def subdivision(a, b, n):
    d = (b-a)/n
    return [a+i*d for i in range(n+1)]

def add_function_curve(f, a, b, n=100):
    xs = subdivision(a, b, n)
    fxs = [f(x) for x in  xs]
    plt.plot(xs, fxs, "r-")

## Téglalap közelítés

In [ ]:
k_values = [k for k in range(1, 11)]
for k in k_values:
    plt.fill_between([k+i for k in k_values for i in range(2)], [math.log(k) for k in k_values for i in range(2)], color='lightblue')

add_function_curve(math.log, 1, 11)


A hiba:

In [ ]:
def primitive_function(x):
    return x*(math.log(x)-1)

def rectangle_error(k):
    return primitive_function(k+1)-primitive_function(k)-math.log(k)

def cummulative_error(n, error_fun=rectangle_error):
    return sum(error_fun(k) for k in range(1, n+1))

In [ ]:
for n in [10, 100, 1000]:
    print(f"{n=}, {cummulative_error(n)=}")

### Javítás, trapéz összeg közelítés

Téglalapok helyett minden egység intervallumon a beírt trapézt  használjuk.

In [ ]:
k_values = [k for k in range(1, 11)]
for k in k_values:
    plt.fill_between(k_values, [math.log(k) for k in k_values], color='lightblue')

add_function_curve(math.log, 1, 10)


In [ ]:
def error_fun(x):
    k, t = divmod(x, 1)
    return math.log(x) - ((1-t)*math.log(k) + t*math.log(k+1))

xs = subdivision(1, 10, 100)
plt.fill_between(xs, [error_fun(x) for x in xs], color='lightblue')
add_function_curve(error_fun, 1, 10, 500)


In [ ]:
def modified_error_fun(x):
    k = x//1
    return k*k*error_fun(x)

for a in [1, 100, 1000, 10000]:
    b = a+10
    xs = subdivision(a, b, 1000)

    plt.fill_between(xs, [modified_error_fun(x) for x in xs], color='lightblue')
    add_function_curve(modified_error_fun, a, b, 1000)
    plt.show()

In [ ]:
def trapezoid_error(k):
    return primitive_function(k+1)-primitive_function(k)-0.5*(math.log(k)+math.log(k+1))


In [ ]:
for n in [10, 100, 1000]:
    print(f"{n = :>4}, {cummulative_error(n, trapezoid_error) = :.8f}")

## Látszik, hogy a hiba lassan nő. Tudunk-e felső becslést adni rá?

$$
    \int_{k}^{k+1} \log x dx = \int_0^1 \log(k+x) dx
$$

A beírt trapéz területe integrállal

$$
    \int_0^1 x\log(k+1)+(1-x)\log(k) dx
$$

Tudjuk-e becsülni a két integrandus különbségét?
$$
    \log(k+x) - (x\log(k+1)+(1-x)\log(k))
$$

Átalakítás mindkét tagból levonunk $\log(k)$-t:
$$
    \log(k+x) - \log(k) - (x\log(k+1)+(1-x)\log(k) -\log(k)) = \log(1+\tfrac{x}{k}) - x\log(1+\tfrac1k)
$$


In [ ]:
xs = subdivision(0, 1, 100)

for k in range(1, 4):
    plt.fill_between(xs, [math.log(k+x)-math.log(k) for x in xs], [math.log(k+1)*x+math.log(k)*(1-x)-math.log(k) for x in xs], color="blue")

plt.grid()

A logaritmus függvény konkáv, a derivált monoton fogy ($1/x$) és egy beírt húr mindig a végponthoz behúzott érintő egyenesek alatt van.

$$
    \log (1+x)\leq x
$$

ezért
$$
    \log(1+x/k) - x \log(1+1/k)\leq \frac xk - x\log(1+1/k) = x(1/k -\log(1+1/k))
$$

de
$$
    \log(1+1/k) = \log\frac{k+1}{k} = -\log\frac{k}{k+1} = -\log(1-1/(k+1)) \geq \frac1{k+1}
$$

Így

$$
    \log(1+x/k) - x \log(1+1/k)\leq x\left(\frac1k-\frac1{k+1}\right)
$$
és
a $k$. intervallumon elkövetett hiba legfeljebb
$$
    \int_{0}^{1} \log(k+x) - ((1-x)\log(k)+x\log(k+1))dx = \int_0^1 x dx \left(\frac1k -\frac1{k+1}\right)
$$
A hibák összege legfeljebb:
$$
    \frac12 \sum_{k=1}^\infty \frac1k -\frac1{k+1} =\frac12
$$

Összefoglalva:

$$
    \log n! = \int_1^n \log x dx + \frac12 \log n + c_n = n(\log(n) - 1) + 1 + \frac12 \log (n) + r_n
$$
ahol $r_n$ a közelítés hibája az első $n$ intervallumon
$$
    r_n = \sum_{k=1}^n \log(1+x/k)-x\log(1+1/k)dx \leq 1/2
$$
$(r_n)$ monoton nő, ezért létezik limesze.

Visszaírva faktoriálisra:

$$
    n! = \sqrt{n}\left(\frac{n}{e}\right)^n c_n
$$
ahol $c_n=e^{1+r_n}\leq e^{3/2}$

Analízisben a Wallis formula következményeként szerepel

$$
\lim c_n = \sqrt{2\pi}
$$

Középiskolai tudást használva is kiszámíthatnánk:
$$
    \sqrt{n}\frac1{2^{2n}}\binom{2n}{n}
    = \sqrt{n} \frac1{2^{2n}} \frac{c_{2n} \sqrt{2n} \left(\frac{2n}{e}\right)^{2n}}{c_n^2 n\left(\frac{n}{e}\right)^{2n}}
    \to \lim_{n\to\infty} \frac{c_{2n}}{c_n^2} = \lim_{n\to\infty}\frac{\sqrt{2}}{c_n}
$$

Az
$$
    I_n = \int_{0}^{\pi/2} \cos^n(x) dx
$$
sorozatot vizsgálva,
$$
\frac1{2^{2n}}\binom{2n}{n}\approx \frac{1}{\sqrt{\pi n}}
$$
adódik,  amiből $\lim_n c_n =\sqrt{2\pi}$.

Ez a nevezetes **Stirling** formula:
$$
 \frac{n!}{\sqrt{2\pi n}\left(\frac{n}{e}\right)^n} \to 1
$$


# Feladatok előadásról


Írjuk meg a faktoriális függvényt rekurzió nélkül.

In [ ]:
def factorial(n):
        pass

Írjunk egy függvényt, amely inputként vár két függvényt és az output az a függvény, amelyik a két input összege.

Azaz ha $f$ és $g$ a két függvény (pl. $f(x)=x+1$, $g(x)=2x+1$), akkor az output legyen az a $h$ függvény, melyre
$h(x) = f(x) + g(x)$.

In [ ]:
def add_two_functions(f, g):
    def h(x):
        pass
    return h

Írjunk egy függvényt, amely inputként vár két függvényt és az output az a függvény, amelyik a két input kompozíciója.

Azaz ha
$f$ és $g$ a két függvény (pl. $f(x)=x+1$, $g(x)=2x+1$), akkor az output legyen az a $h$ függvény, melyre
$h(x) = f(g(x))$.


In [ ]:
def compose(f, g):
    def h(x):
        pass
    return h



## Tesztelés

***Előre érdemes megírni, ez is segít végiggondolni mit is várunk a függvénytől!***

In [ ]:
# uncomment and run if ipytest is not installed
# ! pip install ipytest --quiet

In [ ]:
import ipytest
ipytest.autoconfig()

In [ ]:
%%ipytest

def f(x):
    return x+1

def g(x):
    return 2*x + 1

def test_compose():
    h = compose(f, g)
    for x in [-100, 0, 100, 10000]:
        assert h(x) == f(g(x))
